# Generator 

generator는 values의 시퀀스를 만들어내는 오브젝트다. for문처럼 iterator로 사용할 수도 있고, 혹은 next function을 이용해서 그 다음 value를 가져올 수 있다.(하지만 그 value에 대해서 단 한번만 iterate할 수 있다.) <br>
generator는 함수 안에서 yield 키워드를 사용해서 만들 수 있다. generator function이 호출되면, generator object가 만들어 진다.

In [1]:
def fibonacci_generator():
    a, b = 0, 1
    while True:
        yield a
        a, b = b, a + b

In [3]:
# Print all the numbers of the Fibonacci sequence that are lower than 1000
for i in fibonacci_generator():
    if i > 1000:
        break
    print(i)

0
1
1
2
3
5
8
13
21
34
55
89
144
233
377
610
987


더 간단한 케이스로, generator expression을 이용해서 generator를 만들 수도 있다. 리스트와는 달리, values는 실시간으로 계산된다. (메모리에 저장되지 않는다.)

In [4]:
a = (x * x for x in range(100))

# a is a generator object
print(type(a))

# Sum all the numbers of the generator
print(sum(a))

# There are no elements left in the generator
print(sum(a))

<class 'generator'>
328350
0


# Collections 

collections은 여러 대안적인 데이터 스트럭처를 다룰 수 있는 표준 라이브러리

## 1. Counter 

counter는 elements를 dictionary-key로 저장하면서 그 elements의 수를 카운트한다.

In [5]:
from collections import Counter

a = Counter('blue')
b = Counter('yellow')

print(a)
print(b)
print((a + b).most_common(3))

Counter({'e': 1, 'b': 1, 'u': 1, 'l': 1})
Counter({'l': 2, 'e': 1, 'w': 1, 'o': 1, 'y': 1})
[('l', 3), ('e', 2), ('b', 1)]


## 2. defaultdict 

일반 딕셔너리는 그 key가 없는데 참조하려 하면 error를 내지만, defaultdict는 미리 지정해둔 default value를 리턴한다.

In [7]:
from collections import defaultdict

my_dict = defaultdict(lambda: 'Default Value')
my_dict['a'] = 42

print(type(my_dict))
print(my_dict['a'])
print(my_dict['b'])

<class 'collections.defaultdict'>
42
Default Value


이러한 defaultdict를 이용하면 tree 형태의 데이터 구조를 만들 수 있다.

In [8]:
from collections import defaultdict
import json

def tree():
    """
    Factory that creates a defaultdict that also uses this factory
    """
    return defaultdict(tree)

root = tree()
root['Page']['Python']['defaultdict']['Title'] = 'Using defaultdict'
root['Page']['Python']['defaultdict']['Subtitle'] = 'Create a tree'
root['Page']['Java'] = None

print(json.dumps(root, indent=4))


{
    "Page": {
        "Python": {
            "defaultdict": {
                "Title": "Using defaultdict",
                "Subtitle": "Create a tree"
            }
        },
        "Java": null
    }
}


# itertools 

itertools는 효과적으로 루핑을 돌 수 있게 iterators를 만들어 주는 표준 라이브러리

## 1. permutations 

permutations은 인풋의 가능한 모든 ordering 집합을 생성한다. 3p1

In [13]:
from itertools import permutations

for p in permutations([1,2,3]):
    print(p)

(1, 2, 3)
(1, 3, 2)
(2, 1, 3)
(2, 3, 1)
(3, 1, 2)
(3, 2, 1)


## 2. combinations 

combinations은 인풋의 ordering에 상관없는 집합(콤비네이션)을 생성한다.

In [11]:
from itertools import combinations

for c in combinations([1, 2, 3, 4], 2):
    print(c)

(1, 2)
(1, 3)
(1, 4)
(2, 3)
(2, 4)
(3, 4)


## 3. chain 

chain은 iteration이 가능한 elements를 묶어서 이어준다. 

In [14]:
from itertools import chain

for c in chain(range(3), range(12, 15)):
    print(c)

0
1
2
12
13
14


# Packing/Unpacking 

\* operator(unpack 혹은 splat operator라고도 불리는)는 리스트나 튜플로부터 개별 변수 혹은 인자로 변환하고자 할 때 편리하게 사용할 수 있다.

In [17]:
a, *b, c = [2, 7, 5, 6, 3, 4, 1]
print(a)
print(b)
print(c)

2
[7, 5, 6, 3, 4]
1


In [19]:
a = [1,2,3]
print(a)
print(*a)

[1, 2, 3]
1 2 3


함수의 인자가 이미 리스트나 튜플 안에 있을때, 그들이 list라면 \*args을 dict라면 **kwargs를 사용해서 unpack할 수 있다. 

In [21]:
def repeat(count, name):
    for i in range(count):
        print(name)

print("Call function repeat using a list of arguments:")
args = [4, "cats"]
repeat(*args)

print("Call function repeat using a dictionary of keyword arguments:")
args2 = {'count': 4, 'name': 'cats'}
repeat(**args2)

Call function repeat using a list of arguments:
cats
cats
cats
cats
Call function repeat using a dictionary of keyword arguments:
cats
cats
cats
cats


반대도 가능하다. packing

In [28]:
def f(*args, **kwargs):
    print("Arguments: ", args)
    print("Keyword arguments: ", kwargs)

f(3, 4, 9, foo=42, bar=7,good='z')

Arguments:  (3, 4, 9)
Keyword arguments:  {'bar': 7, 'good': 'z', 'foo': 42}


# Decorator 

decorator는 function을 parameter로 받아서 function을 return하는 function이다. 
예를 들어, 아래 코드를 보면, cache function은 이미 연산해둔 피보나치 숫자들을 기억해두는 함수로 사용된다.

In [31]:
def cache(function):
    cached_values = {}  # Contains already computed values
    def wrapping_function(*args):
        if args not in cached_values:
            # Call the function only if we haven't already done it for those parameters
            cached_values[args] = function(*args)
        return cached_values[args]
    return wrapping_function

@cache
def fibonacci(n):
    print('calling fibonacci(%d)' % n)
    if n < 2:
        return n
    return fibonacci(n-1) + fibonacci(n-2)

print([fibonacci(n) for n in range(1, 9)])

calling fibonacci(1)
calling fibonacci(2)
calling fibonacci(0)
calling fibonacci(3)
calling fibonacci(4)
calling fibonacci(5)
calling fibonacci(6)
calling fibonacci(7)
calling fibonacci(8)
[1, 1, 2, 3, 5, 8, 13, 21]


funtools는 몇몇 decorator를 제공한다. 예를 들어 lru_cache는 같은 인자로 주어진 함수가 호출 되면 가장 최근의 호출결과를 저장해둔다. (시간 절약!)

In [32]:
from functools import lru_cache

@lru_cache(maxsize=None)
def fibonacci(n):
    print('calling fibonacci(%d)' % n)
    if n < 2:
        return n
    return fibonacci(n-1) + fibonacci(n-2)

print([fibonacci(n) for n in range(1, 9)])

calling fibonacci(1)
calling fibonacci(2)
calling fibonacci(0)
calling fibonacci(3)
calling fibonacci(4)
calling fibonacci(5)
calling fibonacci(6)
calling fibonacci(7)
calling fibonacci(8)
[1, 1, 2, 3, 5, 8, 13, 21]


# Context Managers 

Context managers는 자원을 적절히 관리하기 위해 사용된다. 가장 주로 사용되는 context manager는 with open과 같은 형태로 파일을 여는 기능이다. 하지만 대부분의 개발자들은 이를 스스로 만들어서 사용할 수 있다는 것을 모른다. 

실제로 context manager는 \__enter\__와 \__exit\__ 메소드를 가진 클래스이다.

In [34]:
from time import time


class Timer():
    def __init__(self, message):
        self.message = message

    def __enter__(self):
        self.start = time()
        return None  # could return anything, to be used like this: with Timer("Message") as value:

    def __exit__(self, type, value, traceback):
        elapsed_time = (time() - self.start) * 1000
        print(self.message.format(elapsed_time))


with Timer("Elapsed time to compute some prime numbers: {}ms"):
    primes = []
    for x in range(2, 500):
        if not any(x % p == 0 for p in primes):
            primes.append(x)
    print("Primes: {}".format(primes))

Primes: [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499]
Elapsed time to compute some prime numbers: 0.9758472442626953ms


간단하게 사용하기 위해 @contextmanager decorator와 generator function을 사용할 수 도 있다.

In [35]:
from contextlib import contextmanager


@contextmanager
def colored_output(color):
    print("\033[%sm" % color, end="")
    yield
    print("\033[0m", end="")


print("Hello, World!")
with colored_output(31):
    print("Now in color!")
print("So cool.")

Hello, World!
Now in color!
So cool.


In [38]:
print("\033[31mhihi")

hihi
